In [34]:
#
# NCFApp
# New Customers Forecast Application
# Programma creato per il forecast dei clienti
# 

import numpy             as np    # libreria per la gestione dei numeri
import pandas            as pd    # libreria per la gestione del file csv
import seaborn           as sb    # libreria per la costruzione di grafici
import matplotlib.pyplot as plt   # libreria ausiliaria per la costruzione di grafici
import matplotlib
matplotlib.use("TkAgg")

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.figure import Figure

from Tkinter import *
from PIL import ImageTk, Image
import os

#import Tkinter           as tk    # libreria per l'interfaccia grafica
#import PIL                        # libreria per la gestione delle immagini
from statsmodels.tsa.stattools    import acf, pacf, adfuller, arma_order_select_ic
from statsmodels.tsa.seasonal     import seasonal_decompose
from statsmodels.tsa.arima_model  import ARIMA, ARIMAResults
from statsmodels.stats.diagnostic import acorr_ljungbox

def reducto(timeserie) :
# Funzione che taglia la coda della serie storica e tiene i valori piu recenti
# Le serie vengono tagliate ai 4 anni più recenti e ...
# + 12 mesi per la differenziazione stagionale e ...
# + 1 mese per la differenziazione de-trend
    if len(timeserie)>61 :
        tms_object = timeserie[-61:]
    elif len(timeserie)>49 :
        tms_object = timeserie[-49:]
    elif len(timeserie)>37 :
        tms_object = timeserie[-37:]
    return tms_object

def Componi(first_list,second_list) :
    third_list = []
    for first in first_list :
        for second in second_list :
            third_list.append(first+'_'+second)
    return third_list

# Liste di uso comune e dizionario con le declinazioni dei codici

global Tipomisura
Tipomisura       = ('FL','CP','FN','ST')
global Tipomisura_file
Tipomisura_file  = ('FL','CP','ST')
global Tipomisura_fore
Tipomisura_fore  = ('FL','CP')
global Tipocliente
Tipocliente      = ('IMP','PER','AZR','RET','TOTRET','BDT')
global Tipocliente_file
Tipocliente_file = ('IMP','PER','AZR','RET')
global Tipocliente_fore
Tipocliente_fore = ('IMP','PER','AZR','RET')
global Tiposerie
Tiposerie        = ('CONS','FORE','TUTTO')
#Descrittore_serie = [] # FL_IMP, CP_PER ......
#for TM in Tipomisura :
#    for TC in Tipocliente :
#        Descrittore_serie.append(TM+'_'+TC)
#Soggetti_Forecast = [] # elementi di descrittore serie per i quali verr calcolato il forecast
#for TM in Tipomisura[:2] :
#    for TC in Tipocliente[:4] :
#        Soggetti_Forecast.append(TM+'_'+TC)

# Declinazioni
global Decl
Decl = {}
Decl['FL'] = 'FLUSSO LORDO'
Decl['CP'] = 'CLIENTI PERSI'
Decl['FN'] = 'FLUSSO NETTO'
Decl['ST'] = 'STOCK'
Decl['IMP'] = 'IMPRESE'
Decl['PER'] = 'PERSONAL'
Decl['AZR'] = 'AZIENDE RETAIL'
Decl['RET'] = 'PRIVATI RETAIL'
Decl['TOTRET'] = 'TOTALE RETAIL'
Decl['BDT'] = 'TOTALE BDT'
Decl['CONS'] = 'CONSUNTIVO'
Decl['FORE'] = 'FORECAST'
Decl['TUTTO'] = 'CONS.+FORECAST'

global alpha
alpha = 0.05
global mesi_forecast
mesi_forecast = 24



# Carica il dataframe dei dati dal file csv e converte la colonna 'MESE' da stringa a data
# Il dataframe contiene FL,CP e STOCK dei 4 tipi clienti : imprese,personal,aziende retail,privati retail
# i flussi netti sono calcolati per differenza e il totale BDT e retail per somma
global df_sstor
dfp = pd.read_csv('NCFAdati.csv',sep=',',index_col=0, parse_dates=True, infer_datetime_format=True)
df_sstor = pd.DataFrame()
listone = Componi(Tipomisura_file,Tipocliente_file)
for txt in listone : df_sstor[txt] = reducto(dfp[txt])
prima_lista   = Componi(['FN'],Tipocliente_file)
seconda_lista = Componi(['FL'],Tipocliente_file)
terza_lista   = Componi(['CP'],Tipocliente_file)
listone = zip(prima_lista,seconda_lista,terza_lista)
for x,y,z in listone : df_sstor[x] = df_sstor[y] - df_sstor[z]
prima_lista   = Componi(Tipomisura,['TOTRET'])
seconda_lista = Componi(Tipomisura,['AZR'])
terza_lista   = Componi(Tipomisura,['RET'])
listone = zip(prima_lista,seconda_lista,terza_lista) 
for x,y,z in listone : df_sstor[x] = df_sstor[y] + df_sstor[z]
prima_lista   = Componi(Tipomisura,['BDT'])
seconda_lista = Componi(Tipomisura,['IMP'])
terza_lista   = Componi(Tipomisura,['PER'])
quarta_lista  = Componi(Tipomisura,['AZR'])
quinta_lista  = Componi(Tipomisura,['RET'])
listone = zip(prima_lista,seconda_lista,terza_lista,quarta_lista,quinta_lista) 
for x,y,z,w,t in listone : df_sstor[x] = df_sstor[y] + df_sstor[z] + df_sstor[w]+df_sstor[t]
global ultimo_consuntivo
ultimo_consuntivo = max(df_sstor.index)
global index_forecast
index_forecast = pd.date_range(ultimo_consuntivo, periods=1+mesi_forecast, freq='M')
index_forecast = index_forecast[1:]

# Prepara il dizionario dei grafici e lo inizializza    
global Graphics
global Graphics_active
Graphics     = {}
GraphicsSub1 = {}
GraphicsSub2 = {}

        
image_standard = Figure(figsize=(4,2.5),dpi=100) #Image.open('Logobasso.png')
print 'body1',image_standard
grafo = image_standard.add_subplot(111)
print 'body2',image_standard
#image_standard = ImageTk.PhotoImage('RGB',file='Logobasso.png',size=(400,250))
for TM in Tipomisura :
    GraphicsSub2[TM] = image_standard
for TS in Tiposerie  :
    GraphicsSub1[TS] = GraphicsSub2
for TC in Tipocliente :
    Graphics[TC] = GraphicsSub1
            
for TC in Tipocliente :
    for TM in Tipomisura :
        titolo = TM+'_'+TC
        TMS = df_sstor[titolo]
        TMS_rolmean = TMS.rolling(window=12,center=False).mean()
        Graphics[TC]['CONS'][TM] = image_standard  
        
global bb
bb = grafo.plot([1,2,3,4,5,6,7,8],[4,2,7,7,5,3,1,6])
#lim(2,8) #min(newy), max(newy))
#        plt.plot(TMS,label='CONSUNTIVO')
#        plt.plot(TMS_rolmean,label='Media mobile')
#        plt.title(Decl[TM]+'  '+Decl[TC])
#        plt.legend()
#        ax = matplotlib.axes.Axes(plt.figure(),(0,0,400,250))
#        Graphics[TC]['CONS'][TM] = matplotlib.image.AxesImage(ax)

         
#        Graphics[TC]['CONS'][TM] = plt.figure
#        plt.clf

Graphics_active = Graphics['BDT']['CONS']
print 'body3',Graphics_active
      


    
    
def ForecastManager() :
    sequences = ('FL_IMP','CP_IMP','FL_PER','CP_PER','FL_AZR','CP_AZR','FL_RET','CP_RET')
    TMS_value      = []
    TMS_startgraph = []
    TMS_endgraph   = []
    for TMS in sequences :
        TimeSerie = pd.Series(df_sstor[TMS])
        print '\n',TMS,'\n'
        listofforecast = Forecast(TimeSerie, graphic=True)
        TMS_value.append(listofforecast[0])
        TMS_startgraph.append(listofforecast[1])
        TMS_endgraph.append(listofforecast[3])
    
    NCFAfore = pd.concat(TMS_value,axis=1)
    NCFAfore.insert(loc=2,  column='Forecast for FN_IMP',value=NCFAfore['Forecast for FL_IMP']-NCFAfore['Forecast for CP_IMP'])
    NCFAfore.insert(loc=5,  column='Forecast for FN_PER',value=NCFAfore['Forecast for FL_PER']-NCFAfore['Forecast for CP_PER'])
    NCFAfore.insert(loc=8,  column='Forecast for FN_AZR',value=NCFAfore['Forecast for FL_AZR']-NCFAfore['Forecast for CP_AZR'])
    NCFAfore.insert(loc=11, column='Forecast for FN_RET',value=NCFAfore['Forecast for FL_RET']-NCFAfore['Forecast for CP_RET'])
    NCFAfore['Forecast for FL_TOTRET'] = NCFAfore['Forecast for FL_RET']+NCFAfore['Forecast for FL_AZR']
    NCFAfore['Forecast for CP_TOTRET'] = NCFAfore['Forecast for CP_RET']+NCFAfore['Forecast for CP_AZR']
    NCFAfore['Forecast for FN_TOTRET'] = NCFAfore['Forecast for FN_RET']+NCFAfore['Forecast for FN_AZR']
    NCFAfore['Forecast for FL_BDT']    = NCFAfore['Forecast for FL_IMP']+NCFAfore['Forecast for FL_PER']+NCFAfore['Forecast for FL_TOTRET']
    NCFAfore['Forecast for CP_BDT']    = NCFAfore['Forecast for CP_IMP']+NCFAfore['Forecast for CP_PER']+NCFAfore['Forecast for CP_TOTRET']
    NCFAfore['Forecast for FN_BDT']    = NCFAfore['Forecast for FN_IMP']+NCFAfore['Forecast for FN_PER']+NCFAfore['Forecast for FN_TOTRET']
    NCFAfore.to_csv('NCFAfore.csv',sep=';')
    



# Funzione che scrive una stringa nel file di log
def Wlog(stringa=None) :
    file_log.writelines([stringa,'\n'])

    
    # apre il file di log
file_log = open('NCFAlog.txt', mode='w')
Wlog('Aperta sessione di programma')

# Carica il dataframe dei dati dal file csv e converte la colonna 'MESE' da stringa a data
# Il dataframe contiene FL,CP e STOCK dei 4 tipi clienti : imprese,personal,aziende retail,privati retail
# i flussi netti sono calcolati per differenza e il totale BDT e retail per somma
#dfp = pd.read_csv('NCFAdati.csv',sep=',',index_col=0, parse_dates=True, infer_datetime_format=True)

#df_sstor = pd.DataFrame()
#for txt in ['FL_IMP','CP_IMP','ST_IMP','FL_PER','CP_PER','ST_PER','FL_AZR','CP_AZR','ST_AZR','FL_RET','CP_RET','ST_RET'] :
#    df_sstor[txt] = reducto(dfp[txt])

#for x,y,z in zip(['FN_IMP','FN_PER','FN_AZR','FN_RET'],['FL_IMP','FL_PER','FL_AZR','FL_RET'],['CP_IMP','CP_PER','CP_AZR','CP_RET']) :
#    df_sstor[x] = df_sstor[y] - df_sstor[z]

#for x,y,z in zip(['FL_TOTRET','CP_TOTRET','ST_TOTRET','FN_TOTRET'],['FL_AZR','CP_AZR','ST_AZR','FN_AZR'],['FL_RET','CP_RET','ST_RET','FN_RET']) :
#    df_sstor[x] = df_sstor[y] + df_sstor[z]

#for x,y,w,z in zip(['FL_BDT','CP_BDT','ST_BDT','FN_BDT'],['FL_IMP','CP_IMP','ST_IMP','FN_IMP'],['FL_PER','CP_PER','ST_PER','FN_PER'],['FL_TOTRET','CP_TOTRET','ST_TOTRET','FN_TOTRET']) :
#    df_sstor[x] = df_sstor[y] + df_sstor[w] + df_sstor[z]







        #self.imgA = self.imglogo.resize(size=(600,150))
        #self.imgB = PIL.ImageTk.PhotoImage(self.imgA)       #self.canvas1 = tk.Canvas(self,width=600,height=150)
        #self.canvas1.create_image(300,75,image=self.imgA)
        #self.canvas1.grid()
        
#        self.label = tk.Label(text = 'Last consuntive '+str(Lastcons()))
#à        self.label.grid()
        
#        self.quitButton = tk.Button(self, text='Quit', background='darkred', command=self.quit)
#        self.quitButton.grid()
        
#    def quit(self):
#        self.destroy()
#        return 

#quitButton = Button(app, text='Quit', background='darkred', command=quit)
#quitButton.pack()

    #v = StringVar()
    #v.set("L") # initialize

#    for text, mode in MODES:
#        b = Radiobutton(master, text=text,
#                        variable=v, value=mode)
#        b.pack(anchor=W)
#Radiobutton(master, text="One", variable=v, value=1).pack(anchor=W)
#Radiobutton(master, text="Two", variable=v, value=2).pack(anchor=W)


class Application:

    def __init__(self,master):
        #df_sstor = self.CreaDataframe
        #print df_sstor
        #self.df_sstor, self.ultimo_consuntivo, self.index_forecast = self.CreaDataframe
        #self.Graphic_init()
        
        self.frame1 = Frame(master)
        self.frame1.pack(side='top')
        
        self.img_logo = ImageTk.PhotoImage(Image.open("logobanca.png"))
        self.panel_logo = Label(self.frame1,image = self.img_logo, width=400)
        self.panel_logo.pack(side = "left", fill = "both", expand = "yes")
        
        self.pulsante1 = Button(self.frame1, text='Elaborazione FORECAST') 
        self.pulsante1["background"] = "white"
        self.pulsante1.bind('<Button-1>',self.ButtonForecast)
        self.pulsante1.pack(side='bottom')                              

        self.pulsante3 = Button(self.frame1, text='Log') 
        self.pulsante3["background"] = "green"
        self.pulsante3.bind('<Button-1>',self.ButtonLog)
        self.pulsante3.pack(side = 'right')                  
        
        self.pulsante2 = Button(self.frame1, text='Documentazione') 
        self.pulsante2["background"] = "cyan"
        self.pulsante2.bind('<Button-1>',self.ButtonDocumentazione)
        self.pulsante2.pack(side='bottom')                   
        
        self.label1 = Label(self.frame1,text = ' Ultimo consuntivo caricato '+str(ultimo_consuntivo)[0:-12]+' ')
        self.label1.width = 400
        self.label1.pack(side='bottom')
               
        self.frame3a = Frame(master, width=800)
        self.frame3a.pack(side='top')
        
        f = Figure(figsize=(4,2.5), dpi=100)
        self.a = f.add_subplot(111)
        self.a.plot([1,2,3,4,5,6,7,8],[5,6,1,3,8,9,3,5])
        print 'a',self.a
        print 'f',f
        self.imgFL = f
        #self.imgFL = Graphics_active['FL']
        print 'appl1',self.imgFL
        
        self.canvasFL = FigureCanvasTkAgg(self.imgFL, self.frame3a)
        self.canvasFL.show()
        self.canvasFL.get_tk_widget().pack(side='left', fill='both', expand=True)
        #self.canvasFL = Canvas(self.frame3a, width=400, height=250)
        #self.canvasFL.pack(side='left')
        
        #self.imgFL = ImageTk.PhotoImage(Graphics_active['FL'])
        #print self.imgFL
        #self.imageoncanvasFL = self.canvasFL.create_image(0,0,image=self.imgFL)
        
        #self.imgCP = self.Graphics_active['CP']
        self.canvasCP = Canvas(self.frame3a, width=400, height=250)
        self.canvasCP.pack(side='right')
        #self.imageoncanvasCP = self.canvasCP.create_image(0,0,image=self.imgCP)
        
        self.frame3b = Frame(master)
        self.frame3b.pack(side='top')
        
        #self.imgFN = self.Graphics_active['FN']
        self.canvasFN = Canvas(self.frame3b, width=400, height=250)
        self.canvasFN.pack(side='left')
        #self.imageoncanvasFN = self.canvasFN.create_image(0,0,image=self.imgFN)
        
        #self.imgST = self.Graphics_active['ST']
        self.canvasST = Canvas(self.frame3b, width=400, height=250)
        self.canvasST.pack(side='right')
        #self.imageoncanvasST = self.canvasST.create_image(0,0,image=self.imgST)
        
        #self.Refreshgraph('BDT','CONS')
        self.frame4 = Frame(master, width=900)
        self.frame4.pack(side='top')
        # Crea il primo gruppo di pulsanti radio
        radiobase1 = StringVar()
        for code in Tipocliente :
            Radiobutton(self.frame4, text=Decl[code], variable=radiobase1, value=code,indicatoron=0).pack(anchor=W,side='left')
        # Crea il secondo gruppo di pulsanti radio
        radiobase2 = StringVar()
        for code in Tiposerie :
            Radiobutton(self.frame4,background='blue',foreground='white',
                        width=12, text=Decl[code], variable=radiobase2, value=code, indicatoron=0).pack(anchor=W,side='right')
        
        self.frame5 = Frame(master)
        self.frame5.pack(side='top')
        self.label5a = Label(self.frame5,text = ' ')
        self.label5a.pack(fill = "both", expand = "yes")
        self.label5b = Label(self.frame5,text = 'Applicazione per il forecast del flusso clienti - Release 1.0 dicembre 2017 - (C) Gianni Zattoni')
        self.label5b.pack(fill = "both", expand = "yes")
        self.label5c = Label(self.frame5,text = ' ')
        self.label5c.pack(fill = "both", expand = "yes")
        
    def ButtonForecast(self,evento) :
        self.imgFL.clear()
        ax = self.imgFL.add_subplot(111)
        ax = self.a
        ax.set_ylim(2,8)
        ax.set_xlim(1,8)
        #ax.plot = bb
        #ax.plot([1,2,3,4,5,6,7,8],[4,2,7,7,5,3,1,6])
        #print ax
        self.canvasFL.draw()
#        self.pulsante1["text"] = "Waiting..."     
#        self.pulsante1["background"] = "black"
#        ForecastManager()
#        self.pulsante1["text"] = "Elaborazione FORECAST"     
#        self.pulsante1["background"] = "white"
        pass

    def ButtonDocumentazione(self,evento) :
        self.imgFL.clear()
        ax = self.imgFL.add_subplot(111)
        ax.set_ylim(2,8)
        ax.set_xlim(1,8)
        #ax.plot = bb
        ax.plot([1,2,3,4,5,6,7,8],[4,2,7,7,5,3,1,6])
        print ax
        #self.canvasFL = FigureCanvasTkAgg(self.imgFL, self.frame3a)
        #self.canvasFL.itemconfig(self.imgFL)         
        #print 'func img',self.imgFL
        #print 'func grf',Graphics['BDT']['CONS']['FL']
        #self.imgFL = Graphics['BDT']['CONS']['FL']
        #self.canvasFL.figure = self.imgFL
        #self.canvasFL.show()
        #print 'func img',self.imgFL
        self.canvasFL.draw()
        pass
    
    def ButtonLog(self,evento):
        pass
    
    def Refreshgraph(self, tipocliente, tiposerie):
        #imgFL = Graphics[tipocliente][tiposerie]['FL']
        imgFL = ImageTk.PhotoImage(Graphics[tipocliente][tiposerie]['FL'])
    #self.imgCP = ImageTk.PhotoImage(Graphics[tipocliente][tiposerie]['CP'])
    #self.imgFN = ImageTk.PhotoImage(Graphics[tipocliente][tiposerie]['FN'])
    #self.imgST = ImageTk.PhotoImage(Graphics[tipocliente][tiposerie]['ST'])
    #self.canvasFL.image = imgFL
        self.imageoncanvasFL = self.canvasFL.create_image(image=imgFL)
        self.canvasFL.itemconfig(self.imageoncanvasFL, image = self.img_logo)
    #self.canvasCP.image = self.imageCP
    #self.canvasFN.image = self.imageFN
    #self.canvasST.image = self.imageST
#    def Graphic_init(self):
        # Dizionario dei grafici e inizializzazione
        # prima dimensione   IMP PER AZR RET TOTRET BDT
        # seconda dimensione CONS FORE TUTTO
        # terza dimensione   FL CP FN ST
#        self.Graphics     = {}
#        GraphicsSub1 = {}
#        GraphicsSub2 = {}
#        image_standard = ImageTk.PhotoImage('RGB',file='Logobasso.png',size=(400,250))
#        for TM in Tipomisura :
#            GraphicsSub2[TM] = image_standard
#à        for TS in Tiposerie  :
#            GraphicsSub1[TS] = GraphicsSub2
#        for TC in Tipocliente :
#            self.Graphics[TC] = GraphicsSub1
            
#        for TC in Tipocliente :
#            for TM in Tipomisura :
#                titolo = TM+'_'+TC
#                TMS = self.df_sstor[titolo]
#                TMS_rolmean = TMS.rolling(window=12,center=False).mean()
#                plt.plot(TMS,label='CONSUNTIVO')
#                plt.plot(TMS_rolmean,label='Media mobile')
#                plt.title(Decl[TM]+'  '+Decl[TC])
#                plt.legend()
#                self.Graphics[TC]['CONS'][TM] = plt.figure()
#                plt.clf
        
#        self.Graphics_active = self.Graphics['BDT']['CONS']
        
#    def CreaDataframe() :
        # Carica il dataframe dei dati dal file csv e converte la colonna 'MESE' da stringa a data
        # Il dataframe contiene FL,CP e STOCK dei 4 tipi clienti : imprese,personal,aziende retail,privati retail
        # i flussi netti sono calcolati per differenza e il totale BDT e retail per somma
#        dfp = pd.read_csv('NCFAdati.csv',sep=',',index_col=0, parse_dates=True, infer_datetime_format=True)
#        df_sstor = pd.DataFrame()
#        listone = Componi(Tipomisura_file,Tipocliente_file)
#        for txt in listone : df_sstor[txt] = reducto(dfp[txt])
#        prima_lista   = Componi(['FN'],Tipocliente_file)
#        seconda_lista = Componi(['FL'],Tipocliente_file)
#        terza_lista   = Componi(['CP'],Tipocliente_file)
#        listone = zip(prima_lista,seconda_lista,terza_lista)
#        for x,y,z in listone : df_sstor[x] = df_sstor[y] - df_sstor[z]
#        prima_lista   = Componi(Tipomisura,['TOTRET'])
#        seconda_lista = Componi(Tipomisura,['AZR'])
#à        terza_lista   = Componi(Tipomisura,['RET'])
#        listone = zip(prima_lista,seconda_lista,terza_lista) 
#à        for x,y,z in listone : df_sstor[x] = df_sstor[y] + df_sstor[z]
#        prima_lista   = Componi(Tipomisura,['BDT'])
#        seconda_lista = Componi(Tipomisura,['IMP'])
#à        terza_lista   = Componi(Tipomisura,['PER'])
#à        quarta_lista  = Componi(Tipomisura,['AZR'])
#        quinta_lista  = Componi(Tipomisura,['RET'])
#        listone = zip(prima_lista,seconda_lista,terza_lista,quarta_lista,quinta_lista) 
#        for x,y,z,w,t in listone : df_sstor[x] = df_sstor[y] + df_sstor[z] + df_sstor[w]+df_sstor[t]
#        self.df_sstor = df_sstor
#        self.ultimo_consuntivo = max(df_sstor.index)
#        index_forecast = pd.date_range(ultimo_consuntivo, periods=1+mesi_forecast, freq='M')
#        self.index_forecast = index_forecast[1:]
#        print df_sstor
#        return df_sstor #, ultimo_consuntivo, index_forecast
    
#    def Lastcons():
        # Funzione che ritorna l'ultimo consuntivo caricato
#        df_mesi = self.df_sstor.index
#        return max(df_mesi)

root = Tk()
App = Application(root)
root.title('New Customers Forecast Application - rel. 1.0 - (C) Gianni Zattoni')
root.mainloop()

body1 Figure(400x250)
body2 Figure(400x250)
body3 {'CP': <matplotlib.figure.Figure object at 0x7f7ce420f050>, 'FL': <matplotlib.figure.Figure object at 0x7f7ce420f050>, 'FN': <matplotlib.figure.Figure object at 0x7f7ce420f050>, 'ST': <matplotlib.figure.Figure object at 0x7f7ce420f050>}
a Axes(0.125,0.125;0.775x0.755)
f Figure(400x250)
appl1 Figure(400x250)
Axes(0.125,0.125;0.775x0.755)


In [ ]:
TMS = df_sstor['FL_IMP']
a = plt.plot(TMS,label='CONSUNTIVO')
f=plt.figure()
h = matplotlib.axes.Axes(f,(0,0,400,250))
print h
j = matplotlib.image.AxesImage(h)
print j



In [ ]:
# Funzione che data una serie ne calcola il forecast
# l'argomento è la serie storica originale, 
def Forecast(TMS_orig = None, graphic = False):
    
    def test_stationarity(tms = None) :
        # Dickey - Fuller test
        # H0 : la serie non è stazionaria
        # ritorna il p-value
        dftest = adfuller(tms, autolag='AIC')
        Wlog(' ')
        Wlog("Test Dickey-Fuller per verifica stazionarieta'")
        Wlog('Serie temporale     : '+tms.name)
        Wlog("H0 - ipotesi nulla  : La serie non e' stazionaria")
        Wlog('p_value riscontrato : {:05.4f}'.format(dftest[1]))
        if dftest[1]>alpha :
            Wlog("L'ipotesi nulla non si rifiuta - La serie NON e' stazionaria")
        else:
            Wlog("L'ipotesi nulla viene rifiutata - La serie e' stazionaria")
        Wlog(' ')
        return dftest[1]
        
        #print 'Results of Dickey-Fuller Test:'
        #dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
        #for key,value in dftest[4].items():
        #dfoutput['Critical Value (%s)'%key] = value
        #print dfoutput

    def test_whitenoise(tms = None) :   
        # Test Ljung-box sui residui per verifica White noise
        # (H0 : i residui sono WN)
        testWN = acorr_ljungbox(tms,lags=12)
        p_value_min = min(testWN[1])
        Wlog(' ')
        Wlog("Test Ljung-Box per verifica WHITE NOISE")
        Wlog('Serie temporale     : '+tms.name)
        Wlog("H0 - ipotesi nulla  : La serie e' white noise")
        Wlog('p_value riscontrato : {:05.4f}'.format(p_value_min))
        if p_value_min>alpha :
            Wlog("L'ipotesi nulla non si rifiuta - La serie e' White Noise")
        else:
            Wlog("L'ipotesi nulla viene rifiutata - La serie NON e' White Noise")
        Wlog(' ')
        return p_value_min
    
    Wlog(' ')
    Wlog('Iniziata procedura di FORECAST per la serie temporale '+TMS_orig.name)
    Wlog(' ')
    Wlog('Numero di osservazioni presenti      : '+str(len(TMS_orig)))
    
    #Elimina eventuali spazi vuoti
    TMS_orig = TMS_orig.dropna()
    Wlog('Numero di osservazioni post-cleaning : '+str(len(TMS_orig)))
    
    #La serie viene tagliata ai 4 anni più recenti e ...
    # + 12 mesi per la differenziazione stagionale e ...
    # + 1 mese per la differenziazione de-trend
    #if len(TMS_orig)>61 :
    #    TMS_orig = TMS_orig[-61:]
    #elif len(TMS_orig)>49 :
    #    TMS_orig = TMS_orig[-49:]
    #elif len(TMS_orig)>37 :
    #    TMS_orig = TMS_orig[-37:]
    #else :
    #    raise 0
    #Wlog('Numero di osservazioni che verranno prese in considerazione (più recenti) : '+str(len(TMS_orig)))
    
    #Calcola ed espone media mobile e deviazione standard su 12 periodi
    TMS_rolmean = TMS_orig.rolling(window=12,center=False).mean()
    #TMS_rolstd  = TMS_orig.rolling(window=12,center=False).std()
        
    plt.plot(TMS_orig, label='Original Data')
    plt.plot(TMS_rolmean, label='Rolling Mean')
    #plt.plot(TMS_rolstd, label='Rolling Standard Dev.')
    plt.title('Original series and Rolling Mean')
    plt.legend()
    start_graph = plt.figure()
    if graphic : plt.show()
    print start_graph
    # differenziazione per la stagionalita'
    TMS_diff_seas = TMS_orig - TMS_orig.shift(12) #effetto stagionalita' eliminato
    TMS_effetto_trend = TMS_diff_seas / 12        #effetto trend mensile
    #trend mensile medio sull'ultimo anno
    effetto_trend = np.mean(TMS_effetto_trend[-12:].values)
    
    Wlog('Ricalcolata serie temporale depurata degli effetti stagionali')
    Wlog("Ricalcolata serie temporale depurata dell'effetto trend")
    Wlog('Effetto trend mensile (media degli ultimi 12 mesi)'+str(effetto_trend))
    
    decomposition = seasonal_decompose(TMS_orig,two_sided=False)
    TMS_trend     = decomposition.trend.dropna()
    TMS_seas      = decomposition.seasonal.dropna()
    TMS_resid     = decomposition.resid.dropna()
    TMS_resid.name = TMS_orig.name + "_residual"
    
    plt.subplot(411)
    plt.plot(TMS_orig, label='Original')
    plt.legend(loc='best')
    plt.subplot(412)
    plt.plot(TMS_trend, label='Trend')
    plt.legend(loc='best')
    plt.subplot(413)
    plt.plot(TMS_seas,label='Seasonality')
    plt.legend(loc='best')
    plt.subplot(414)
    plt.plot(TMS_resid, label='Residuals')
    plt.legend(loc='best')
    plt.tight_layout()
    decompose_graph = plt.figure()
    if graphic : plt.show()
    
    # Ricerca il numero di differenziazioni sino a giungere alla stazionarietà
    max_diff  = 5
    TMS_for_test = TMS_resid
    rank_diff = 0
    
    Wlog('Test per la verifica della stazionarietà sulla serie destagionalizzata e detrendizzata')
    
    while (test_stationarity(TMS_for_test) > alpha) and (rank_diff < max_diff) :
        rank_diff += 1
        TMS_for_test = TMS_for_test - TMS_for_test.shift()
        TMS_for_test = TMS_for_test.dropna()
        TMS_for_test.name = TMS_resid.name + '('+str(rank_diff)+')'
    
    Wlog('Numero di differenziazioni da attuare per rendere la serie stazionaria : '+str(rank_diff))
    Wlog('Se uguale a zero   : valore ammissibile ma piuttosto raro')
    Wlog('Se uguale a uno    : valore standard e diffuso')
    Wlog('Se uguale a due    : possono esserci trend anomali (almeno quadratici)')
    Wlog('Se maggiore di due : valori anomali - serie temporale imprevedibile, anche se converge rischio di inferenza distorta')
    
    # se la serie è già stazionaria prova a vedere se si tratta di un white noise
    if rank_diff == 0 :
        if test_whitenoise(TMS_resid) > alpha : 
            whitenoise = True
        else :
            whitenoise = False
    else :
        whitenoise = False
        
    if whitenoise :
        Wlog('Serie temporale uguale al white noise - nessuna ulteriore informazione catturabile')
        TMS_predict = pd.Series(0,index=df_sstor.index)
        TMS_forecast = pd.Series(0,index=index_forecast)
    else :
        # rappresentazione di ACF e PACF
        lag_acf = acf(TMS_for_test, nlags=15)
        lag_pacf = pacf(TMS_for_test, nlags=15, method='ols')
        #Plot ACF: 
        plt.subplot(121) 
        plt.plot(lag_acf)
        plt.axhline(y=0,linestyle='--',color='gray')
        plt.axhline(y=-1.96/np.sqrt(len(TMS_for_test)),linestyle='--',color='gray')
        plt.axhline(y=1.96/np.sqrt(len(TMS_for_test)),linestyle='--',color='gray')
        plt.title('Autocorrelation Function')
        #Plot PACF:
        plt.subplot(122)
        plt.plot(lag_pacf)
        plt.axhline(y=0,linestyle='--',color='gray')
        plt.axhline(y=-1.96/np.sqrt(len(TMS_for_test)),linestyle='--',color='gray')
        plt.axhline(y=1.96/np.sqrt(len(TMS_for_test)),linestyle='--',color='gray')
        plt.title('Partial Autocorrelation Function')
        plt.tight_layout()
        acfpacf_graph = plt.figure()
        if graphic : plt.show()
    
        autoregressive      = TMS_resid
        autoregressive_diff = TMS_for_test
    
        criteria_ic = arma_order_select_ic(autoregressive_diff)
        rank_AR = criteria_ic.bic_min_order[0]
        rank_MA = criteria_ic.bic_min_order[1]
        Wlog('Modello autoregressivo stimato : ARIMA('+str(rank_AR)+','+str(rank_diff)+','+str(rank_MA)+')')
    
        results_AR = ARIMA(autoregressive,order=(rank_AR,rank_diff,rank_MA)).fit(trend='c',disp=-1)
    
        TMS_predict = results_AR.fittedvalues
 
        #print results_AR.summary()
        #plt.plot(autoregressive_diff)
        #plt.plot(TMS_predict, color='red')
        #plt.title('RSS: %.4f'% sum((TMS_predict.dropna()-autoregressive_diff)**2))
        #plt.show()
    
        forecast = results_AR.forecast(steps=mesi_forecast)
        #print forecast
        TMS_forecast    = pd.Series(forecast[0],index=index_forecast)
        forecast_stderr = pd.Series(forecast[1],index=index_forecast)
   
    # FORECAST

    new_TMS_predict = TMS_predict + TMS_trend + TMS_seas

    new_TMS_trend = pd.Series(effetto_trend,index=index_forecast)
    new_TMS_trend[0] = new_TMS_trend[0] + TMS_trend[-1]
    new_TMS_trend = new_TMS_trend.cumsum()
    
    mask_seas = TMS_seas[-12:]
    new_TMS_seas = pd.Series(0,index=index_forecast)
    index_base = 0
    ranges = range(0,len(index_forecast))
    for index_base in ranges :
        index_mask = index_base % 12
        new_TMS_seas[index_base] = mask_seas[index_mask]
    
    # Serie di forecast finale
    
    new_TMS_forecast = TMS_forecast + new_TMS_trend + new_TMS_seas
    
    plt.plot(TMS_orig,        color='blue', label='Valori osservati')
    plt.plot(new_TMS_predict, color='red',  label='Valori predetti')
    plt.plot(new_TMS_forecast,color='green',label='Valori forecast')
    plt.legend(loc='best')
    end_graph = plt.figure()
    if graphic : plt.show()
    
    
    new_TMS_forecast.name = 'Forecast for '+TMS_orig.name
    
    #print pd.concat([TMS_orig,TMS_trend,TMS_seas,TMS_resid],axis=1)
       #df_fore = pd.DataFrame.from_items([('value',TMS_forecast),('std error',forecast_stderr)])
    return new_TMS_forecast, start_graph, decompose_graph, end_graph

In [ ]:
print df_sstor.head()

In [ ]:
class Application(tk.Frame):
    def __init__(self, master=None):
        tk.Frame.__init__(self, master)
        self.grid()
        self.createWidgets()

    def createWidgets(self):
        
        self.img = ImageTk.PhotoImage(Image.open("Logo.png"))
        self.panel = Label(image = self.img)
        self.panel.pack(side = "bottom", fill = "both", expand = "yes")

        #self.imglogo = PIL.Image.open('Logo.png')
        #self.imgA = self.imglogo.resize(size=(600,150))
        #self.imgB = PIL.ImageTk.PhotoImage(self.imgA)
        #self.canvas1 = tk.Canvas(self,width=600,height=150)
        #self.canvas1.create_image(300,75,image=self.imgA)
        #self.canvas1.grid()
        
        self.label = tk.Label(text = 'Last consuntive '+str(Lastcons()))
        self.label.grid()
        
        self.quitButton = tk.Button(self, text='Quit', background='darkred', command=self.quit)
        self.quitButton.grid()
        
    def quit(self):
        self.destroy()
        return 

root = Application()
root.master.title('New Customers Forecast Application - rel. 1.0 - (C) Gianni Zattoni')
root.mainloop()


In [ ]:
imA = PIL.Image.new(mode='RGB',size=(100,120),color='green')
imA

In [ ]:
imglogo = PIL.Image.open('Logo.jpg')
imB = imglogo
#imB = PIL.ImageTk.PhotoImage(imglogo)
label = tk.Label(image=imB)
label.grid()

In [ ]:
# Frame principale
class Application(tk.Frame):
    def __init__(self, master=None):
        tk.Frame.__init__(self, master)
        self.grid()
        self.createWidgets()

    def createWidgets(self):
        
        self.imglogo = PIL.Image.open('Logo.jpg')
        self.imgA = self.imglogo.resize(size=(600,150))
        #self.imgB = PIL.ImageTk.PhotoImage(self.imgA)
        self.canvas1 = tk.Canvas(self,width=600,height=150)
        #self.canvas1.create_image(300,75,image=self.imgB)
        self.canvas1.grid()
        
        self.label = tk.Label(text = 'Last consuntive '+str(Lastcons()))
        self.label.grid()
        
        self.quitButton = tk.Button(self, text='Quit', command=self.quit)
        self.quitButton.grid()
        
    def quit(self):
        self.destroy()
        return 

app = Application()
app.master.title('New Customers Forecast Application - rel. 1.0 - (C) Gianni Zattoni')
app.mainloop()


In [ ]:
help('PIL.ImageTk')

In [ ]:
# Funzione che prepara un grafico e lo salva in una figura
def SimplePlot(SeriePandas=None,LabelofSerie=None):
    plt.plot(SeriePandas,label=LabelofSerie)
    plt.legend()
    plt.show()
    return plt.figure()

In [ ]:
# create the canvas, size in pixels
canvas = tk.Canvas(width=500, height=200, bg='black')

# pack the canvas into a frame/form
canvas.grid() #expand=YES, fill=BOTH)

# load the .gif image file
logo = PIL.Image.open("Logo.jpg")
gif1 = PIL.ImageTk.PhotoImage(logo)

# put gif image on canvas
# pic's upper left corner (NW) on the canvas is at x=50 y=10
canvas.create_image(50, 10, image=gif1, anchor=N)

In [ ]:

# prova di visualizzazione del grafico

#sb.set(style="ticks")
#pos = rs.randint(-1, 2, (20, 5)).cumsum(axis=1)
#pos -= pos[:, 0, np.newaxis]
#step = np.tile(range(5), 20)
#walk = np.repeat(range(20), 5)

# Initialize a grid of plots with an Axes for each walk
#grid = sb.FacetGrid(d_somma, col="walk", hue="walk", col_wrap=5, size=1.5)

# Draw a horizontal line to show the starting point
#grid.map(plt.axhline, y=0, ls=":", c=".5")

# Draw a line plot to show the trajectory of each random walk
#grid.map(plt.plot, "step", "position", marker="o", ms=4)

# Adjust the tick positions and labels
#grid.set(xticks=np.arange(5), yticks=[-3, 3],
#         xlim=(-.5, 4.5), ylim=(-3.5, 3.5))

# Adjust the arrangement of the plots
#grid.fig.tight_layout(w_pad=1)



print type(fig)
graphic = tk.Canvas(app)
q = graphic.create_image(x=1, y=1, image=fig) #canvas.get_tk_widget().grid(row=1,column=24)
canvas.draw()


# create the canvas, size in pixels
#canvas = Canvas(width=300, height=200, bg='black')

# pack the canvas into a frame/form
#canvas.pack(expand=YES, fill=BOTH)

# load the .gif image file
#gif1 = PhotoImage(file='small_globe.gif')

# put gif image on canvas
# pic's upper left corner (NW) on the canvas is at x=50 y=10
#canvas.create_image(50, 10, image=gif1, anchor=N